## 1. 데이터 불러오기

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('0. 서울시 부동산 실거래가 정보.csv', encoding='utf-8')

C:\Users\user\AppData\Local\Temp\ipykernel_29808\3996529006.py:1: DtypeWarning: Columns (7,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('0. 서울시 부동산 실거래가 정보.csv', encoding='utf-8')


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2619094 entries, 0 to 2619093
Data columns (total 21 columns):
 #   Column            Dtype  
---  ------            -----  
 0   접수연도              int64  
 1   자치구코드             int64  
 2   자치구명              object 
 3   법정동코드             int64  
 4   법정동명              object 
 5   지번구분              float64
 6   지번구분명             object 
 7   본번                object 
 8   부번                float64
 9   건물명               object 
 10  계약일               int64  
 11  물건금액(만원)          int64  
 12  건물면적(㎡)           float64
 13  토지면적(㎡)           float64
 14  층                 float64
 15  권리구분              object 
 16  취소일               float64
 17  건축년도              float64
 18  건물용도              object 
 19  신고구분              object 
 20  신고한 개업공인중개사 시군구명  object 
dtypes: float64(7), int64(5), object(9)
memory usage: 419.6+ MB


## 2. 데이터 전처리

단위 변환 및 열 선택

In [4]:
# 면적 평 수로 변환
df['건물면적'] = df['건물면적(㎡)'].map(lambda x: x*0.3025)

# 가격 억으로 변환
df['물건금액'] = df['물건금액(만원)'].map(lambda x: x*0.0001)

#필요한 열만 선택
df=df[['자치구명','법정동명','건물명','건축년도','층','건물용도','건물면적','물건금액']]

df.head()

,자치구명,법정동명,건물명,건축년도,층,건물용도,건물면적,물건금액
0,강남구,역삼동,역삼더빌,2012.0,4.0,연립다세대,3.802425,1.5
1,강남구,역삼동,역삼더빌,2012.0,4.0,연립다세대,3.802425,1.5
2,강남구,자곡동,강남블루지움,2017.0,6.0,오피스텔,14.432275,3.7
3,강남구,자곡동,강남 더샵라르고,2014.0,10.0,오피스텔,7.753075,2.0
4,강남구,도곡동,도곡렉슬,2006.0,19.0,아파트,25.711895,28.5


결측치 관리

In [5]:
# 새로운 데이터 프레임에 결측치 확인 
df.isna().sum()

자치구명         0
법정동명         0
건물명     207359
건축년도      7876
층       207308
건물용도         0
건물면적         0
물건금액         0
dtype: int64

In [6]:
#건물명은 결측치값이 분석에 영향을 주지 않아서 처리 하지 않고 진행. 
#건축년도는 추측할 수 있는 단서가 없으므로 결측치,이상치를 제거한다.
df = df.dropna(subset=['건축년도'])

#고유값 확인
df['건축년도'].unique()

array([2012., 2017., 2014., 2006., 2015., 2002., 1992., 2005., 1991.,
       1998., 2003., 1997., 1982., 1999.,    0., 1979., 2018., 2013.,
       2000., 2004., 2016., 2011., 2019., 1994., 1983., 2020., 1993.,
       1980., 2001., 1987., 1990., 2008., 2021., 1985., 2022., 1984.,
       1977., 1976., 1988., 1995., 2009., 1978., 2010., 2007., 1996.,
       1986., 2023., 1989., 1981., 1974., 1975., 1973., 1900., 1972.,
       1971., 1968., 1969., 1967., 1966., 1970., 1962., 1961., 1963.,
       1901., 1965., 1964., 1960., 1955., 1958., 1949., 1950., 1959.,
       1957., 1945., 1956., 2024., 1941., 1946., 1940., 1928., 1927.,
       1933., 1947., 1954., 1908., 1909., 1951., 1944., 1948., 1912.,
       1936., 1953., 1952., 1937., 1931., 1943., 1942., 1938., 1939.,
       1922., 1915., 1935., 1910., 1932., 1907., 1921., 1934., 1916.,
       1929., 1926., 1923., 1930., 1917., 1911., 1905., 1918.,  970.,
       1920., 1925., 1914.])

In [7]:
#0인 행 제거 
df = df[df['건축년도'] != 0]
df=df.reset_index(drop=True)

#결측치 재확인 
df.isna().sum()

자치구명         0
법정동명         0
건물명     202863
건축년도         0
층       202812
건물용도         0
건물면적         0
물건금액         0
dtype: int64

In [8]:
# 층 결측치 확인 
df[df['층'].isna()]['건물용도'].unique()
# 이 결측치에 해당하는 건물용도를 확인한 결과 모두 '단독 다가구'이다.

array(['단독다가구'], dtype=object)

In [9]:
#'층'이 결측값인 행
print(len(df[df['층'].isna()]))

#'건물용도'가 '단독다가구'인 행
print(len(df.query('건물용도=="단독다가구"')))

# 두 개가 202812개로 동일
# 단독 다가구에 대한 층 처리가 안 되어서 결측치가 모두 단독 다가구에 해당. 
# 단독다가구는 법적으로 3층 이하 건물로 취급. 
# 따라서 단독다가구에 대한 층을 1층으로 바꿈.

202812
202812


In [10]:
# '층'을 1로 채움
df['층'].fillna(1, inplace=True)

# 인덱스 리셋
df=df.reset_index(drop=True)

# 고유값 확인. 결측치 사라짐!
df['층'].unique()

C:\Users\user\AppData\Local\Temp\ipykernel_29808\1239124027.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['층'].fillna(1, inplace=True)


array([ 4.,  6., 10., 19., 11.,  2.,  3.,  7., 15.,  9.,  1., 45.,  8.,
       12.,  5., 13., 14., 29., 18., 17., 22., 39., 20., 25., 16., 23.,
       24., -1., 30., 26., 50., 21., 49., 41., 46., 55., 54., 58., 53.,
       27., 34., 64., 33., 68., 38., 31., 42., 28., 36., 43., 62., 35.,
       32., 40., 47., 56., 63., 52., 60., 57., 44., 66., 48., 59., 61.,
       67., 51., 37., 65., -2., 69., -3., -4.])

In [11]:
#결측치 처리가 모두 완료된 것을 알 수 있음. 
df.isna().sum()

자치구명         0
법정동명         0
건물명     202863
건축년도         0
층            0
건물용도         0
건물면적         0
물건금액         0
dtype: int64

In [12]:
# 결측치 처리가 잘 되었는지 확인 하기 위한 코드 
# 건물용도가 단독다가구인 행의 '층'열 값은?!
df[(df['건물용도'] == '단독다가구')]['층'].unique() 

array([1.])

유용한 열 생성

In [13]:
#건축년도를 구분하기 위해 반복문 사용을 사용하여 '건축년도구분'이라는 새로운 열 추가  
for i in range(len(df)):
    if df.loc[i, '건축년도']<1980:
        df.loc[i, '건축년도구분'] = '1979년 이하'
    elif df.loc[i, '건축년도']<1990:
        df.loc[i, '건축년도구분'] = '1980년대'
    elif df.loc[i, '건축년도']<2000:
        df.loc[i, '건축년도구분'] = '1990년대'  
    elif df.loc[i, '건축년도']<2010:
        df.loc[i, '건축년도구분'] = '2000년대'    
    elif df.loc[i, '건축년도']<2020:
        df.loc[i, '건축년도구분'] = '2010년대'        
    else:
        df.loc[i, '건축년도구분'] = '2020년대'

In [14]:
#층을 구분하기 위해 반복문 사용을 사용하여 '층 구분'이라는 새로운 열을 추가 
for i in range(len(df)):
    if df.loc[i, '층']<0:
        df.loc[i, '층구분'] = '00층 이하'
    elif df.loc[i, '층']<=5:
        df.loc[i, '층구분'] = '05층이하'
    elif df.loc[i, '층']<=10:
        df.loc[i, '층구분'] = '10층이하'
    elif df.loc[i, '층']<=15:
        df.loc[i, '층구분'] = '15층이하'
    elif df.loc[i, '층']<=20:
        df.loc[i, '층구분'] = '20층이하'
    elif df.loc[i, '층']<=30:
        df.loc[i, '층구분'] = '30층이하'
    elif df.loc[i, '층']<=40:
        df.loc[i, '층구분'] = '40층이하'
    else:
        df.loc[i, '층구분'] = '50층이하'

In [15]:
#건물면적을 구분하기 위해 반복문 사용을 사용하여 '건물면적구분'이라는 새로운 열을 추가 
for i in range(len(df)):
    if df.loc[i, '건물면적']<10:
        df.loc[i, '건물면적구분'] = '10평 미만'
    elif df.loc[i, '건물면적']<20:
        df.loc[i, '건물면적구분'] = '10평대'
    elif df.loc[i, '건물면적']<30:
        df.loc[i, '건물면적구분'] = '20평대'
    elif df.loc[i, '건물면적']<40:
        df.loc[i, '건물면적구분'] = '30평대'
    elif df.loc[i, '건물면적']<50:
        df.loc[i, '건물면적구분'] = '40평대'
    elif df.loc[i, '건물면적']<60:
        df.loc[i, '건물면적구분'] = '50평대'
    else:
        df.loc[i, '건물면적구분'] = '60평대 이상'

In [16]:
df.columns = df.columns.str.replace(' ', '')
file_name = '1. 전처리 후 서울시 부동산 실거래가 정보.csv'
df.to_csv(file_name, index=False)